# RSA Attack using Coppersmith Theorem

Step 1: Initialise RSA values

In [1]:
# initialise my RSA
p = 12869795083300688777180977539261293509742299186625597947007346052371885560376413412969538812952531788746738888696190568941522462811532669027187276982231027
q = 12148353615556056154472437834933029851320451773077215044227637191346624898202059936109696194890997673818589947427245092370775886593279153368858540169203579
N = 156346821631681477401124926750495286740792129458553032234494386688081624853288356506937772456803832811784712148005027115830528920515787519711656110036850378116016773959834119229061429110509500800467962987062370248424826070636629116147537042363934287867297000405976659980678829286591117944009130578389173245633
e = 5

print(f"Prime p : {p}\n")

print(f"Prime q : {q}\n")
print(f"Large N : {N}\n")
print(f"Encryption Key: {e}")
gcd_output = gcd((p-1)*(q-1),e)
print(f"GCD check : {gcd_output}")

Prime p : 12869795083300688777180977539261293509742299186625597947007346052371885560376413412969538812952531788746738888696190568941522462811532669027187276982231027

Prime q : 12148353615556056154472437834933029851320451773077215044227637191346624898202059936109696194890997673818589947427245092370775886593279153368858540169203579

Large N : 156346821631681477401124926750495286740792129458553032234494386688081624853288356506937772456803832811784712148005027115830528920515787519711656110036850378116016773959834119229061429110509500800467962987062370248424826070636629116147537042363934287867297000405976659980678829286591117944009130578389173245633

Encryption Key: 5
GCD check : 1


Step 2: Generate stereotyped message  
According to Coppersmith, if missing only N^1/e of message then considered small root  
Using theorem, can find small roots mathematically

In [2]:
# Stereotyped message
length_N = 1024

unknown_m = "RAIN"

um_ascii = [ord(i) for i in unknown_m]
u_m = ''.join(map(str, um_ascii))
length_m = int(u_m).bit_length()
print(f"Want to find: {u_m}")

# attach "unknown_m" to arbitrary message
K = ZZ(u_m)
Kdigits = K.digits(2)
M = [0]*length_m + [1]*(length_N-length_m);   # generate "unknown_m" zeros and "1024-unknown_m" ones
for i in range(len(Kdigits)):                 # insert "unknown_m" into least sig bits
    M[i] = Kdigits[i]

# generate ciphertext
ZmodN = Zmod(N)
M = ZZ(M, 2)                                  # convert M into integers
c = ZmodN(M)^e
print(f"ciphertext : {c}")

Want to find: 82657378
ciphertext : 58074113415568888121781007933081000015716841721347441993874012269825067084697161709454922978847443675473060301388643827209049161432989366251494206694502186485938684334807981807739290744441320829510639531250723921237833312375046715388591105089033451363033491349486954003644562328160622724911574948092367309929


In [3]:
# Defining the Equation
R.<x> = PolynomialRing(ZmodN)               # x is part of integer ring mod N
eqn = (2^length_N - 2^length_m + x)^e - c   # equation as defined by coppersmith

In [4]:
# Variables to adjust
beta = 0.9                                    # must satisfy 0 < beta <= 1
deg = eqn.degree()
epsilon = beta / 7                            # as defined by coppersmith theorem
copp_m = ceil(beta**2 / (deg * epsilon))      # computing m as defined by coppersmith for later polynomials
copp_t = floor(deg * copp_m * ((1/beta) - 1)) # computing t as defined by coppersmith for later polynomials
copp_X = ceil(N**((beta**2/deg) - epsilon))   # boundary X, which root must not exceed

In [5]:
# Coppersmith function
def coppersmith_theorem(eqn, deg, N, beta, copp_m, copp_t, copp_X):
    
    lat_length = copp_t + (deg*copp_m)
    
    # check if variables respect all the boundaries for X and both Howgrave-Graham and LLL
    det_LLL = RR(N^((1/2) * (deg*copp_m*(copp_m + 1))) * copp_X^((1/2)*(lat_length * (lat_length - 1))))
    LLL_bound = RR(2^((lat_length - 1)/4) * det_LLL^(1/lat_length))
    HG_bound = RR(N^(beta*copp_m) / sqrt(lat_length))
    if LLL_bound < HG_bound:
        print("Howgrave-Graham and LLL boundary respected")
    else:
        print("Howgrave-Graham and LLL boundary not respected. Try other variables")
        
    X_bound = RR((1/2) * N^(((2*beta*copp_m)/(lat_length-1)) - ((deg*copp_m*(copp_m+1))/(lat_length*(lat_length-1)))))
    if copp_X <= X_bound:
        print("X boundary respected")
    else:
        print("X boundary not respected. Try other variables")

    # applying Howgrave-Graham Theorem, we change polynomial ring of x from (int mod N) to just int
    eqn_z = eqn.change_ring(ZZ)
    x = eqn_z.parent().gen()

    # generate the polynomials to be used in lattice
    g = []
    for w in range(copp_m):                                           
        for u in range(deg):                                           
            g.append(((x*copp_X)^u) * N^(copp_m-w) * eqn_z(x*copp_X)^w)   # g polynomial
    for w in range(copp_t):
        g.append(((x*copp_X)^w) * eqn_z(x*copp_X)^copp_m)               # h polynomial
    
    # create lattice for LLL
    #lat_length = copp_t + (deg*copp_m)
    lati = Matrix(ZZ, lat_length)                                     # square matrix with lat_length width
    
    # fill up the lattice
    for w in range(lat_length):
        for u in range(w+1):
            lati[w, u] = g[w][u]

    # apply the LLL algorithm
    lati = lati.LLL()
    
    #check condition det(L) < N^beta*m*n
    lat_det = lati.det()
    
    
    # take out all the shortest poly to test their root
    new_polynomial = 0
    for w in range(lat_length):
        new_polynomial += (x^w)*lati[0,w]/copp_X^w

    # solve for the root
    short_roots = new_polynomial.roots()

    # test roots
    roots = []
    for root in short_roots:
        if root[0].is_integer():
            result = eqn_z(ZZ(root[0]))
            if gcd(N, result) >= N^beta:
                roots.append(ZZ(root[0]))

    return roots

In [6]:
roots = coppersmith_theorem(eqn, deg, N, beta, copp_m, copp_t, copp_X)

print(f"we want to find: {unknown_m}")

# convert root using ascii
root = str(roots[0])
root_ascii = [int(root[w:w+2]) for w in range(0, len(root), 2)]
answer = ''.join([chr(q) for q in root_ascii])
print(f"we found: {answer}")

Howgrave-Graham and LLL boundary respected
X boundary respected
we want to find: RAIN
we found: RAIN
